<a href="https://colab.research.google.com/github/lizbiz0510/CORNEJO_A4/blob/main/Cornejo_A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [41]:
import tensorflow as tf
import string

In [42]:
with open("The-Alchemist-PDF.txt") as f:
    str_txt = f.read()
    #print(str_txt)
data=str_txt[319:]
#data=str_txt[1699900:]
#print(data)

In [43]:
def clean_text(doc):
    tokens=doc.split()
    table=str.maketrans('','',string.punctuation)
    tokens=[w.translate(table) for w in tokens]
    tokens=[word for word in tokens if word.isalpha()]
    tokens=[word.lower() for word in tokens]
    return tokens
tokens=clean_text(data)
print(tokens[:50])

['the', 'boys', 'name', 'was', 'santiago', 'dusk', 'was', 'falling', 'as', 'the', 'boy', 'arrived', 'with', 'his', 'herd', 'at', 'an', 'abandoned', 'church', 'the', 'roof', 'had', 'fallen', 'in', 'long', 'ago', 'and', 'an', 'enormous', 'sycamore', 'had', 'grown', 'on', 'the', 'spot', 'where', 'the', 'sacristy', 'had', 'once', 'stood', 'he', 'decided', 'to', 'spend', 'the', 'night', 'there', 'he', 'saw']


In [44]:

print(len(tokens))

print(len(set(tokens)))

38484
3286


In [45]:

length = 60+1
lines=[]
for i in range(length,len(tokens)):
    seq = tokens[i-length:i]
    line= ' '.join(seq)
    lines.append(line)
n = int(len(lines)*0.9)
testset = lines[n:]
lines = lines[:n]
print(len(lines), len(testset))

34580 3843


In [46]:
from tensorflow.keras.preprocessing.text import Tokenizer as tk
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [47]:

#converitng the text to integer for model injestion
tokenizer=tk()
tokenizer.fit_on_texts(lines)
sequences=tokenizer.texts_to_sequences(lines)
 
sequences=np.array(sequences)
x,y=sequences[:,:-1],sequences[:,-1]

In [48]:
x[0]

array([   1,  180,  391,    8, 1224, 1697,    8, 3095,   22,    1,   11,
        616,   19,   12, 3094,   21,   62, 1695,  955,    1, 1694,    9,
        689,   10,  267,  421,    3,   62,  806, 1691,    9, 1690,   33,
          1, 3090,   91,    1, 3089,    9,  215,  615,    4,  204,    2,
        498,    1,  157,   31,    4,  150,    2,   13,    7,   40,    1,
         57,  496,  135,    1, 1222])

In [49]:
y[0]

1698

In [50]:
tokenizer.word_index
print(len(tokenizer.word_index))

3095


In [51]:
vocab_size=len(tokenizer.word_index)+1
print(vocab_size)

3096


In [52]:
y=to_categorical(y,num_classes=vocab_size)

In [53]:
seq_length=x.shape[1]

In [54]:
#building LSTM Model

In [55]:
model=Sequential()
model.add(Embedding(vocab_size,60,input_length=seq_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))

In [56]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 60, 60)            185760    
                                                                 
 lstm_2 (LSTM)               (None, 60, 100)           64400     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 3096)              312696    
                                                                 
Total params: 653,356
Trainable params: 653,356
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [58]:
model.fit(x,y,batch_size=100,epochs=100)

Epoch 1/100
346/346 [==============================] - 21s 49ms/step - loss: 6.2085 - accuracy: 0.0883
Epoch 2/100
346/346 [==============================] - 7s 21ms/step - loss: 5.7694 - accuracy: 0.0986
Epoch 3/100
346/346 [==============================] - 4s 12ms/step - loss: 5.5133 - accuracy: 0.1109
Epoch 4/100
346/346 [==============================] - 5s 15ms/step - loss: 5.3281 - accuracy: 0.1305
Epoch 5/100
346/346 [==============================] - 4s 12ms/step - loss: 5.1929 - accuracy: 0.1383
Epoch 6/100
346/346 [==============================] - 4s 13ms/step - loss: 5.0869 - accuracy: 0.1463
Epoch 7/100
346/346 [==============================] - 4s 12ms/step - loss: 4.9955 - accuracy: 0.1508
Epoch 8/100
346/346 [==============================] - 4s 12ms/step - loss: 4.9106 - accuracy: 0.1584
Epoch 9/100
346/346 [==============================] - 5s 13ms/step - loss: 4.8334 - accuracy: 0.1615
Epoch 10/100
346/346 [==============================] - 4s 12ms/step - loss: 4.76

In [59]:
#lets take a random line  
seed_text=lines[12343]


In [60]:
import numpy as np
"""
generate_text_seq() generates n_words number of words after the given seed_text. We are going to pre-process the seed_text before 
predicting. We are going to encode the seed_text using the same encoding used for encoding the training data. Then we are going to
convert the seed_textto 50 words by using pad_sequences(). Now we will predict using model.predict_classes(). After that we will search
the word in tokenizer using the index in y_predict
"""
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    temp = model.predict(encoded, verbose=0)
    #y_predict = model.predict_classes(encoded)
    y_predict = np.argmax(temp, axis=-1)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text) + '\n'

In [61]:
#now lets predict 5 examples for the seed word


print('Example 1:')
seed_text = lines[np.random.randint(0, len(lines)-1)]
print('Seed text: ', seed_text, '\n')
print("AI generated: ", generate_text_seq(model, tokenizer, seq_length, seed_text, 100))

print('Example 2:')
seed_text = lines[np.random.randint(0, len(lines)-1)]
print('Seed text: ', seed_text, '\n')
print("AI generated: ", generate_text_seq(model, tokenizer, seq_length, seed_text, 100))

print('Example 3:')
seed_text = lines[np.random.randint(0, len(lines)-1)]
print('Seed text: ', seed_text, '\n')
print("AI generated: ", generate_text_seq(model, tokenizer, seq_length, seed_text, 100))

print('Example 4:')
seed_text = lines[np.random.randint(0, len(lines)-1)]
print('Seed text: ', seed_text, '\n')
print("AI generated: ", generate_text_seq(model, tokenizer, seq_length, seed_text, 100))

print('Example 5:')
seed_text = lines[np.random.randint(0, len(lines)-1)]
print('Seed text: ', seed_text, '\n')
print("AI generated: ", generate_text_seq(model, tokenizer, seq_length, seed_text, 100))

Example 1:
Seed text:  an enormous sycamore had grown on the spot where the sacristy had once stood he decided to spend the night there he saw to it that all the sheep entered through the ruined gate and then laid some planks across it to prevent the flock from wandering away during the night there were no wolves in the region but once an 

AI generated:  animal had strayed during the land the boy suspected that the universe needs to read the book he had been working for the pink in the desert the boy was sad and he had worked hard for a year in the desert disobedience means death he had been forced to appear you easier for to the room of the sunrise and says that the boy was tempted to be rude and move to the wind although the vision of the seemingly endless hours between sunrise and dusk and he quickly busy the englishmans eyes lit with him the boy said

Example 2:
Seed text:  in the hookah and inhaled deeply ive had this shop for thirty years i know good crystal from bad and everyth

In [64]:
loss, acc = 0,0
for i in testset[0:10]:
  encoded = tokenizer.texts_to_sequences([i])[0]
  encoded = pad_sequences([encoded], maxlen = seq_length, truncating='pre')
  temp = model.evaluate(encoded)
  loss += temp[0]
  acc += temp[1]

print("loss: ", loss/len(testset))
print("accuracy: ", acc/len(testset))

1/1 [==============================] - 0s 25ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
loss:  0.0
accuracy:  0.0
